In [65]:
import os
import sys
sys.path.insert(0, '../gofher')

import itertools

from galaxy import galaxy
from spin_parity import read_spin_parity_galaxies_label_from_csv
from sparcfire import read_sparcfire_galaxy_csv, get_ref_band_and_gofher_params
from panstarrs import visualize_panstarrs, create_panstarrs_csv, PANSTARRS_REF_BANDS_IN_ORDER, PANSTARRS_BANDS_IN_ORDER
from sdss import visualize_sdss, create_sdss_csv, SDSS_REF_BANDS_IN_ORDER, SDSS_BANDS_IN_ORDER
from gofher import run_gofher_on_galaxy_with_fixed_gofher_parameters, normalize_array #TODO: remove last one
from file_helper import check_if_folder_exists_and_create, write_csv

In [66]:
use_panstarrs = True

visualize_gals=True
make_csv=False

the_bulge_disk_r = 0.0625
the_bulge_disk_string = str(the_bulge_disk_r).replace(".","")

In [67]:
if use_panstarrs:
    ##path_to_input = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\original\\galaxies\\" #PANSTARRS
    path_to_input = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog-data\\panstarrs" #for test
else:
    path_to_input = "C:\\Users\\school\\Desktop\\cross_id\\sdss_mosaic_construction" #SDSS

if use_panstarrs:
    #output_folder_name = 'gofher_panstarrs_sparcfire_test_r_0125' #PANSTARRS
    #output_folder_name = 'gofher_panstarrs_sparcfire_sans_u_r_{}'.format(the_bulge_disk_string) #PANSTARRS
    output_folder_name = 'sparcfire_{}'.format(the_bulge_disk_string) #PANSTARRS
else:
    #output_folder_name = 'gofher_sdss_sparcfire_test_r_05' #SDSS
    output_folder_name = 'gofher_sdss_sparcfire_r_{}'.format(the_bulge_disk_string) #SDSS

#folder_name = "table3"
folder_name = "figure9"
#path_to_output = "C:\\Users\\school\\Desktop\\gofher_output_refactor"
#path_to_output = "E:\\grad_school\\research\\spin_parity_panstarrs"
path_to_output = "C:\\Users\\school\\Desktop" #for test

def get_fits_path(name,band):
    """the file path of where existing fits files can be found"""
    return os.path.join(path_to_input,folder_name,name,"{}_{}.fits".format(name,band))

def get_galaxy_list():
    """the list of galaxy names to run on"""
    return os.listdir(os.path.join(path_to_input,folder_name))

def get_save_vis_path(name):
    """the file path specifying where to save the visualizitaion"""
    return os.path.join(path_to_output,output_folder_name,folder_name,"{}.png".format(name))

def get_csv_out_path():
    """the file path specifying where to save the ouput csv"""
    return os.path.join(path_to_output,output_folder_name,"{}.csv".format(folder_name))

def get_color_image_path(name):
    return os.path.join(path_to_input,folder_name,name,"{}_color.jfif".format(name))

def get_dark_side_csv_path():
    csv_path = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\table_info\\csv_format_of_table\\"
    pa = "3" #folder_name.strip()[-1]
    return os.path.join(csv_path,"table_{}.csv".format(pa))

In [68]:
def setup_output_directories():
    output_path = os.path.join(path_to_output, output_folder_name)
    check_if_folder_exists_and_create(output_path)

    output_folder_path = os.path.join(path_to_output, output_folder_name, folder_name)
    check_if_folder_exists_and_create(output_folder_path)

In [69]:
def run_with_sparcfire(name, fits_path, sparcfire_bands, save_vis_path='', dark_side_label='', color_image_path='',bulge_disk_r=1.0):
    """run gofher on a single sdss galaxy"""
    the_gal = galaxy(name,dark_side_label)

    bands_in_oder = PANSTARRS_BANDS_IN_ORDER if use_panstarrs else SDSS_BANDS_IN_ORDER
    ref_bands_in_order = PANSTARRS_REF_BANDS_IN_ORDER if use_panstarrs else SDSS_REF_BANDS_IN_ORDER

    for band in bands_in_oder:
        the_gal.construct_band(band,fits_path(name,band))

    the_ref_band, the_sparcfire_derived_params = get_ref_band_and_gofher_params(sparcfire_bands,ref_bands_in_order,bulge_disk_r)
    if the_ref_band == None or the_sparcfire_derived_params == None: return

    the_gal.ref_band = the_ref_band
    the_band_pairs = list(itertools.combinations(bands_in_oder, 2))

    the_gal = run_gofher_on_galaxy_with_fixed_gofher_parameters(the_gal,the_band_pairs,the_sparcfire_derived_params)
    if save_vis_path != '':
        #save_vis_path = 'C:\\Users\\school\\Desktop\\for_wayne_03_07_2024\\NGC2841.png' #TODO: delte (for wayne test)
        if use_panstarrs:
            visualize_panstarrs(the_gal,save_vis_path,color_image_path)
        else:
            visualize_sdss(the_gal,save_vis_path)
    return the_gal

In [70]:
import copy
import numpy as np

def get_dict_for_side(the_gal,el_mask,side,valid):
    area_to_consider = np.logical_and(np.logical_and(el_mask,side),valid)

    band_data_dict = dict()
    for band in the_gal.bands:
        normed = normalize_array(copy.deepcopy(the_gal.bands[band].data),el_mask)
        band_data_dict[band] = normed[area_to_consider]
    return band_data_dict

def output_pixels(the_gal):
    el_mask = the_gal.create_ellipse()
    pos_mask, neg_mask = the_gal.create_bisection()


    valid_pixel_mask = np.ones(el_mask.shape,dtype=bool)

    for band in the_gal.bands:
        valid_pixel_mask = np.logical_and(valid_pixel_mask,copy.deepcopy(the_gal[band].valid_pixel_mask))
    
    pos_side_dict = get_dict_for_side(the_gal,el_mask,pos_mask,valid_pixel_mask)
    neg_side_dict = get_dict_for_side(the_gal,el_mask,neg_mask,valid_pixel_mask)

    #print(pos_side_dict['g'][0:10])
    #print(neg_side_dict['g'][0:10])
    
    the_header = list(pos_side_dict.keys())
    
    rows = []
    for i in range(len(pos_side_dict[the_header[0]])):
        the_row = []
        for band in the_header:
            the_row.append(pos_side_dict[band][i])
        the_row.append(1)
        rows.append(the_row)
    for i in range(len(neg_side_dict[the_header[0]])):
        the_row = []
        for band in the_header:
            the_row.append(neg_side_dict[band][i])
        the_row.append(0)
        rows.append(the_row)
    the_header.append("side")
    csv_path = "C:\\Users\\school\\Desktop\\for_wayne_03_07_2024\\{}\\{}.csv".format(folder_name,the_gal.name)
    write_csv(csv_path,the_header,rows)


In [71]:
def make_diff_image_example_for_paper(the_gal):
    #https://walmsley.dev/posts/typesetting-mnras-figures
    #https://support.microsoft.com/en-us/office/add-a-font-b7c5f17c-4426-4b53-967f-455339c564c1
    #error - font not appearing: https://stackoverflow.com/a/26106170
    ## delet file at: /mnt/c/Users/school/.matplotlib

    #""""""
    import matplotlib.pyplot as plt
    from gofher import normalize_array
    from visualize import create_color_map_class
    import numpy as np

    SMALL_SIZE = 9
    MEDIUM_SIZE = 9
    BIGGER_SIZE = 9

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)
    plt.rc('font', family='Nimbus Roman No9 L')

    fig, ax = plt.subplots(nrows=2,ncols=2,figsize=(10/3, 4))  # height can be anything
    ##diff = the_gal.get_band_pair('r-y').diff_image
    diff = the_gal.get_band_pair('r-z').diff_image

    data = the_gal['r'].data - the_gal['z'].data
    to_diff_mask = the_gal.create_ellipse()
    mask_out_it = np.logical_not(to_diff_mask)
    
    ##bad_norm_diff = data * to_diff_mask##normalize_array(data,to_diff_mask)
    bad_norm_diff = normalize_array(data,to_diff_mask)

    diff[mask_out_it] = -np.Inf #to make background same color
    bad_norm_diff[mask_out_it] = -np.Inf #to make background same color
    #diff = bad_norm_diff

    shape = diff.shape
    cent = int(shape[0]/2),int(shape[1]/2)
    size = int(shape[0]/5),int(shape[1]/5)
    top_left = cent[0] - size[0]
    bottom_right = cent[0] + size[0]

    the_mask = the_gal[the_gal.ref_band].valid_pixel_mask
    pos_mask,neg_mask = the_gal.create_bisection()
    #m, s = np.mean(data[the_mask]), np.std(data[the_mask])
    
    cmap = create_color_map_class(pos_mask,neg_mask,np.logical_and(to_diff_mask,the_mask))

    import matplotlib.image as mpimg
    #pa_color = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\galaxies\\panstarrs\\{}\\{}\\{}_color.jfif".format(folder_name,the_gal.name,the_gal.name)
    pa_color = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog-data\\panstarrs\\figure9\\NGC3367\\NGC3367_color.jfif"
    image = mpimg.imread(pa_color)
    
    #ax[0].set_title('(a) color:')
    #ax[0].imshow(image)
    #ax[0].set_axis_off()

    ax[0,0].set_title('(a) color:')
    ax[0,0].imshow(image)
    ax[0,0].set_axis_off()
    
    ax[0,1].set_title('(b) reference band:')
    ref = the_gal['i'].data
    m, s = np.mean(ref[to_diff_mask]), np.std(ref[to_diff_mask])

    ax[0,1].imshow(ref[top_left:bottom_right,top_left:bottom_right],origin='lower',cmap='plasma')
    ax[0,1].imshow(cmap[top_left:bottom_right,top_left:bottom_right], origin= 'lower',alpha=0.5)
    ax[0,1].set_axis_off()

    
    ax[1,0].set_title('(c) difference:')
    im2 = ax[1,0].imshow(diff[top_left:bottom_right,top_left:bottom_right],origin='lower',cmap='plasma')
    ax[1,0].set_axis_off()

    ax[1,1].set_title('(d) r-y:')
    data[mask_out_it] = -np.Inf #to make background same color
    im2 = ax[1,1].imshow(data[top_left:bottom_right,top_left:bottom_right],origin='lower',cmap='plasma')
    ax[1,1].set_axis_off()
    fig.tight_layout()

    
    pa = "E:\\grad_school\\research\\spin_parity_panstarrs\\paper_writing\\figure_maker\\{}_{}_alt_norm_diff_r_z.png".format(the_gal.name,output_folder_name)
    fig.savefig(pa, dpi=300)


In [72]:
if use_panstarrs:
    ##path_to_sparcfire_csv = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\original\\output_from_running\\SpArcFiRe\\{}\\galaxy.csv".format(folder_name) #PANSTARRS
    path_to_sparcfire_csv = "C:\\Users\\school\\Desktop\\ngc3367_galaxy.csv" ##for test
else:
    path_to_sparcfire_csv = "C:\\Users\\school\\Desktop\\cross_id\\sdss_mosaic_construction\\SpArcFiRe_output\\{}\\G.out\\galaxy.csv".format(folder_name) #SDSS
sparcfire_galaxy_csv = read_sparcfire_galaxy_csv(path_to_sparcfire_csv)

dark_side_labels = read_spin_parity_galaxies_label_from_csv(get_dark_side_csv_path())
if folder_name == "table2" and "IC 2101" in dark_side_labels:
    dark_side_labels["IC2101"] = dark_side_labels["IC 2101"]

setup_output_directories()

the_gals = []
i = 1
#print(sparcfire_galaxy_csv.keys())
for name in sparcfire_galaxy_csv:
    #name = 'IC1199'#TODO: delte (for wayne test)
    #name = 'NGC157'#TODO: delte (for wayne test)
    #name = 'NGC2841'#TODO: delte (for wayne test)
    print(i,name)
    i += 1
    ##try:
    if True:
        sparcfire_bands = sparcfire_galaxy_csv[name]

        save_vis_path = ''
        if visualize_gals:
            save_vis_path=get_save_vis_path(name)

        paper_dark_side_label = dark_side_labels[name]
    
        the_gal = run_with_sparcfire(name, get_fits_path, sparcfire_bands, save_vis_path=save_vis_path,dark_side_label=paper_dark_side_label,color_image_path=get_color_image_path(name),bulge_disk_r=the_bulge_disk_r)
        make_diff_image_example_for_paper(the_gal)
        the_gals.append(the_gal)
        #output_pixels(the_gal)
        #break
    ##except:
    else:
        print("Error when running on",name)
        #break

if make_csv:
    if use_panstarrs:
        the_band_pairs = list(itertools.combinations(PANSTARRS_BANDS_IN_ORDER, 2)) #PANSTARRS
        create_panstarrs_csv(the_gals,the_band_pairs,get_csv_out_path())
    else:
        the_band_pairs = list(itertools.combinations(SDSS_BANDS_IN_ORDER, 2)) #SDSS
        create_sdss_csv(the_gals,the_band_pairs,get_csv_out_path())

1 NGC3367
